In [1]:
import pandas as pd
import numpy as np

In [2]:
def unboundness(rt): #Checks the unboundness
    proceed = False
    if rt.all() < 0:
        print('The problem is infeasible')
        proceed = True
    return proceed

In [3]:
def cycling(rt,vtlb): #Resolves cycling using Bland's rule
    dum = np.extract(rt == np.inf, rt)
    dum = rt[rt == np.min(dum)].index
    dum1 = [x for x in dum if x not in vtlb]
    return min(dum1)

In [4]:
def check_cycling(rt, dum, vtlb): # Checks for cycling
    count = 0
    for i in dum:
        if (i == np.inf):
            count += 1
    if count == len(dum): #if cycling exists
        return cycling(rt,vtlb)
    else:
        dum = rt[rt == np.min(dum)].index #Resolves the conflict by choosing the varaible with minimum index
        dum1 = [x for x in dum if x not in vtlb]
        return min(dum1)

In [5]:
def row_operations(tab, ib, inb, v2b, v2nb, basis,vtlb): #Row operations on the tableau
    dum = tab.loc[v2nb,v2nb]/tab.loc[v2nb,v2b] #pivoting the variable to basis 
    tab.loc[v2nb,:] = dum * tab.loc[v2nb,:]
    for i in ib + ['z']: # row operations for the pivoted row
        if i != v2nb:
            dum = -tab.loc[i,v2b]/tab.loc[v2nb,v2b]
            tab.loc[i,:] += (dum * tab.loc[v2nb,:])
    dum = basis.index(v2nb) #updating the basis,non basis indices
    basis[dum] = v2b
    inb.remove(v2b)
    inb.append(v2nb)
    vtlb.append(v2nb)
    tab.index = ['z'] + basis
    ib = basis
    return tab, ib, inb,vtlb

In [6]:
def simplex(tab, ib, inb,vtlb):
    basis = ib
    ib = sorted(ib)
    inb = sorted(inb)
    find_pivot = tab.loc['z',inb] #Find the variable to enter the basis
    dum = np.extract(find_pivot >= 0 , find_pivot)
    v2bi = find_pivot[find_pivot == np.max(dum)].index
    v2b = min(v2bi)
    inter = []
    for i in ib: #Find the variable to leave the basis
        if tab.loc[i,v2b]>0:
            inter.append(i)
    rt = tab.loc[inter, 'B'] / tab.loc[inter, v2b]
    if unboundness(rt):
        return tab, ib, inb
    else:
        dum = np.extract(rt >= 0, rt)
        v2nb = check_cycling(rt, dum,vtlb)
        tab, ib, inb,vtlb = row_operations(tab, ib, inb, v2b, v2nb,basis,vtlb)
        print("Variable to basis:", v2b)
        print("Variable to non basis:",v2nb)
        return tab, ib, inb,vtlb

In [7]:
def artificial_pivot(tab,ib,inb,vtlb): # Special case: when artificial variables are present in basis after reaching optimal value in phase 1
    basis = ib
    ib = sorted(ib)
    inb = sorted(inb)
    for i in inb: #choosing the first non zero variable to enter the basis
        if tab.loc['z',i]!=0:
            v2b = i
            break
    rt = tab.loc[ib,'B']/tab.loc[ib,v2b]
    if unboundness(rt):
        return tab, ib, inb
    else:
        dum = np.extract(rt >= 0, rt)
        v2nb = check_cycling(rt, dum,vtlb) #non basis variable exits the basis
        tab, ib, inb,vtlb = row_operations(tab, ib, inb, v2b, v2nb,basis,vtlb)
        print("Variable to basis:", v2b)
        print("Variable to non basis:",v2nb)
        return tab, ib, inb,vtlb

In [8]:
def phase1(tableau,basis_var,non_basis_var,artificial_var,C): #the phase 1
    #Initializing the tableau for phase 1
    tableau.loc['z', basis_var] = 0
    for i in non_basis_var+['B']:
        tableau.loc['z', i] = sum(tableau.loc[artificial_var, i])
    print('Tableau for phase 1 \n',tableau)
    # Condition check to perform the pivoting operations 
    for i in non_basis_var:
        if tableau.loc['z',i]>0:
            condition1 = True
            break
    vtlb1 = [] # This variable keeps the record of the variable that entered the basis -> left the basis -> re-enters the basis
    print('The Pivoting operations done to achieve optimal value is: \n')
    while (condition1):
        tableau, basis_var, non_basis_var,vtlb1 = simplex(tableau,basis_var,non_basis_var,vtlb1)
        for i in non_basis_var:
            if tableau.loc['z',i]>0:
                condition1 = True
                break
            else:
                condition1 = False
    print('\n')
    if tableau.loc['z','B']==0: #checking the optimality of phase 1 
        print('optimality is achieved proceed to phase 2 \n')
        # If there any artificial variables that are present in the basis
        print('Pivoting operations done to remove the artificial variables \n')
        while (len([x for x in artificial_var if x in basis_var ])>0):
            a=[x for x in artificial_var if x in basis_var ]
            b = [x for x in (tableau.columns[:-1]) if x not in artificial_var]
            ''' 
            This is where we'll deal with redundancy in the problem. After the phase 1 optimality is achieved and if 
            artificial variables are present in the basis with corresponding artificial RHS = 0 and that the only artificial
            variable present in the basis, that row can be deleted.
            '''
            if (tableau.loc[basis_var,'B'].all()==0) and (tableau.loc[a,b].all()==0).all(): 
                for i in basis_var:
                    if tableau.loc[i,'B']==0:
                        tableau = tableau.drop([i])
                        basis_var.remove(i)
                        print('x%d is eliminated and this is where the redudancy in the problem is dealt \n'%(i))
            else:
                tableau, basis_var, non_basis_var,vtlb1 = artificial_pivot(tableau,basis_var,non_basis_var,vtlb1)#cycling to move the artificial variables out of basis
        print('\n')
        print('Tableau after phase 1: \n',tableau)
        # Converting the tableau from phase 1 -> phase 2
        tableau = tableau.drop(columns=artificial_var)
        dum = np.zeros((1,len(slack_var)+1))
        dum = np.append(C,dum,axis=1)
        tableau.loc['z',:] = dum
        for i in basis_var:
            if tableau.loc['z',i] != 0:
                dum = tableau.loc['z',i]
                tableau.loc['z',:] -= dum*tableau.loc[i,:]
        tableau.loc['z',:] = -1*tableau.loc['z',:]
        print('the tableau for phase 2 \n',tableau)
        return tableau,basis_var,non_basis_var
    else:
        print('The LP is infeasible \n')
        return tableau,basis_var,non_basis_var
    

In [9]:
def phase2(tableau,basis_var,non_basis_var,artificial_var=[]): #the phase 2
    #condition to check for positive reduced cost
    for i in non_basis_var:
        if tableau.loc['z',i]>0:
            condition = True
            break
    non_basis_var = [x for x in non_basis_var if x not in artificial_var]
    vtlb2 = []
    print('The Pivoting operations done to achieve optimal value is: \n')
    while (condition):
        tableau, basis_var, non_basis_var,vtlb2 = simplex(tableau,basis_var,non_basis_var,vtlb2)
        for i in non_basis_var:
            if tableau.loc['z',i]>0:
                condition = True
                break
            else:
                condition = False
    return tableau, basis_var, non_basis_var

In [10]:
# input the problem in minimization format
typeproblem=1 #if maximization problem change this to -1
A = np.array([[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,1,1,1,1,1,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
              [1,0,0,0,0,1,0,0,0,0,1,0,0,0,0],
              [0,1,0,0,0,0,1,0,0,0,0,1,0,0,0],
              [0,0,1,0,0,0,0,1,0,0,0,0,1,0,0],
              [0,0,0,1,0,0,0,0,1,0,0,0,0,1,0],
              [0,0,0,0,1,0,0,0,0,1,0,0,0,0,1]])
b = np.array([[10],[20],[15],[7],[11],[9],[10],[8]])
C = np.array([[58.5,68.3,45,55,63.5,65.3,74.8,55,49,56,59,61.3,63,58.8,47]])
C = typeproblem*C
'''
For constraint type:
 if <= or < write 1
 if >= or > write -1
 if = write 0
'''
ConsType = np.array([1,1,1,-1,-1,-1,-1,-1])
# ConsType = np.array([0,0,0,0,0,0,0,0])

In [11]:
n_const,n_var = A.shape #The parameters

In [12]:
#getting the indices of artificial, basis and slack variables
slack_var = []
Iden_loc = [0]*n_const
artificial_var = []

In [13]:
# If there are negative b values then change the sign of the values in corresponding row
for i in range(n_const):
    if (b[i]<0):
        A[i,:] = -1*A[i,:]
        b[i] = -1*b[i]
        ConsType[i] = -1*ConsType[i]

In [14]:
# Adding the slack and Identity variables into the list and adding the new variables into A
for i in range(n_const):
    if (ConsType[i]>0):
        n_var = n_var + 1
        dum = np.zeros((n_const,1))
        A = np.append(A,dum,axis=1)
        slack_var.append(n_var-1)
        Iden_loc[i]=(n_var-1)
        A[i,n_var-1] = 1
    elif (ConsType[i]<0):
        n_var = n_var + 1
        dum = np.zeros((n_const,1))
        A = np.append(A,dum,axis=1)
        slack_var.append(n_var-1)
        A[i,n_var-1] = -1

In [15]:
# Adding the artificial and Identity variables into the list
for i in range(n_const):
    if (ConsType[i]<0 or Iden_loc[i]==0):
        n_var = n_var + 1
        dum = np.zeros((n_const,1))
        A = np.append(A,dum,axis=1)
        artificial_var.append(n_var-1)
        Iden_loc[i]=(n_var-1)
        A[i,n_var-1] = 1

In [16]:
print('the slack variables are:',slack_var)
print('the artificial variables are:',artificial_var)
print('the initial basis variables are:',Iden_loc)

the slack variables are: [15, 16, 17, 18, 19, 20, 21, 22]
the artificial variables are: [23, 24, 25, 26, 27]
the initial basis variables are: [15, 16, 17, 23, 24, 25, 26, 27]


In [17]:
basis_var = Iden_loc
non_basis_var = [x for x in range(n_var) if x not in basis_var] #Getting the non basis variable
print('The non basis variable are:', non_basis_var)

The non basis variable are: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 18, 19, 20, 21, 22]


In [18]:
#converting the obatined the matrices into a tableau format
tableau = pd.DataFrame(A)
dum = np.zeros((1, n_var - C.shape[1]))
dum = np.append(C, dum, axis = 1)
dum = pd.DataFrame(dum)
tableau = pd.concat([dum, tableau])
tableau.index = range(n_const+1)
dum = np.append(np.zeros((1, 1)), b, axis  = 0)
dum = pd.DataFrame(dum)
dum
tableau['B'] = dum
tableau.index = ['z'] + basis_var
print('The converted matrices into Tableau format is: \n',tableau)

The converted matrices into Tableau format is: 
        0     1     2     3     4     5     6     7     8     9  ...   19   20  \
z   58.5  68.3  45.0  55.0  63.5  65.3  74.8  55.0  49.0  56.0  ...  0.0  0.0   
15   1.0   1.0   1.0   1.0   1.0   0.0   0.0   0.0   0.0   0.0  ...  0.0  0.0   
16   0.0   0.0   0.0   0.0   0.0   1.0   1.0   1.0   1.0   1.0  ...  0.0  0.0   
17   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...  0.0  0.0   
23   1.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0  ...  0.0  0.0   
24   0.0   1.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0  ... -1.0  0.0   
25   0.0   0.0   1.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0  ...  0.0 -1.0   
26   0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   1.0   0.0  ...  0.0  0.0   
27   0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   1.0  ...  0.0  0.0   

     21   22   23   24   25   26   27     B  
z   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  
15  0.0  0.0  0.0  0.0  0.0  0.0  0.0  10.0  
16

In [19]:
# Checking for the presence of artificial variables 
if (len(artificial_var)==0):
    print('Proceed to phase two')
    tableau,basis_var,non_basis_var=phase2(tableau,basis_var,non_basis_var,artificial_var)
    ph2=0
else:
    print('start phase 1')
    tableau,basis_var,non_basis_var=phase1(tableau,basis_var,non_basis_var,artificial_var,C)
    ph2=1

start phase 1
Tableau for phase 1 
       0    1    2    3    4    5    6    7    8    9  ...   19   20   21   22  \
z   1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ... -1.0 -1.0 -1.0 -1.0   
15  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
16  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  0.0  0.0  0.0   
17  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
23  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
24  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ... -1.0  0.0  0.0  0.0   
25  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0 -1.0  0.0  0.0   
26  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0 -1.0  0.0   
27  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0 -1.0   

     23   24   25   26   27     B  
z   0.0  0.0  0.0  0.0  0.0  45.0  
15  0.0  0.0  0.0  0.0  0.0  10.0  
16  0.0  0.0  0.0  0.0  0.0  20.0  
17  0.0  

In [20]:
if ph2==1:
    tableau,basis_var,non_basis_var=phase2(tableau,basis_var,non_basis_var,artificial_var)

The Pivoting operations done to achieve optimal value is: 

Variable to basis: 0
Variable to non basis: 3
Variable to basis: 9
Variable to non basis: 6
Variable to basis: 2
Variable to non basis: 4
Variable to basis: 16
Variable to non basis: 15
Variable to basis: 5
Variable to non basis: 7
Variable to basis: 14
Variable to non basis: 10


In [21]:
print('The Final tableau is: \n', tableau)

The Final tableau is: 
       0    1    2     3     4    5    6    7    8    9  ...   14   15   16  \
z   0.0 -4.8  0.0 -12.8 -14.3  0.0 -4.5 -3.2 -0.0  0.0  ...  0.0 -6.8  0.0   
0   1.0  1.0  0.0   1.0   1.0  0.0  0.0 -1.0  0.0  0.0  ...  0.0  1.0  0.0   
5   0.0 -1.0  0.0  -1.0  -1.0  1.0  0.0  1.0  0.0  0.0  ...  0.0 -1.0  0.0   
2   0.0  0.0  1.0   0.0   0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0  0.0   
8   0.0  0.0  0.0   1.0   0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
9   0.0  1.0  0.0   0.0   1.0  0.0  1.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
14  0.0 -1.0  0.0   0.0   0.0  0.0 -1.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
11  0.0  1.0  0.0   0.0   0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
16  0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  1.0   

     17    18    19    20    21    22       B  
z  -9.0 -65.3 -70.3 -51.8 -49.0 -56.0  2431.6  
0   0.0   0.0   0.0   1.0   0.0   0.0     1.0  
5   0.0  -1.0   0.0  -1.0   0.0   0.0     6.0  
2  

In [22]:
print('The objective values are: \n')
for i in tableau.columns:
    if i in basis_var:
        print('x%d : %f \n'%(i,tableau.loc[i,'B']))
    elif i in non_basis_var:
        print('x%d : %f \n'%(i,0))
    else:
        print('\n')
        print('The objective function value is: %f'%(typeproblem*tableau.loc['z',i]))

The objective values are: 

x0 : 1.000000 

x1 : 0.000000 

x2 : 9.000000 

x3 : 0.000000 

x4 : 0.000000 

x5 : 6.000000 

x6 : 0.000000 

x7 : 0.000000 

x8 : 10.000000 

x9 : 4.000000 

x10 : 0.000000 

x11 : 11.000000 

x12 : 0.000000 

x13 : 0.000000 

x14 : 4.000000 

x15 : 0.000000 

x16 : 0.000000 

x17 : 0.000000 

x18 : 0.000000 

x19 : 0.000000 

x20 : 0.000000 

x21 : 0.000000 

x22 : 0.000000 



The objective function value is: 2431.600000
